In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm
# For Arrow3D
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
# For structure visualization
from ase import Atoms
from ase.visualize import view
from ase.io.vasp import read_vasp

In [2]:
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        super().__init__((0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def do_3d_projection(self, renderer=None):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, self.axes.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        return np.min(zs)

In [3]:
class TrajectoryOnLattice:
    def __init__(self, poscar, xdatcar, interval=100, target='O'):
        # save arguments
        if os.path.isfile(poscar):
            self.poscar = poscar
        else:
            print(f"{poscar} does not exist!")
            exit()
        if os.path.isfile(xdatcar):
            self.xdatcar = xdatcar
        else:
            print(f"{xdatcar} does not exist!")
            exit()
        self.interval, self.target = interval, target

        # Available parameters
        self.lattice = None             # lattice vectors
        self.lat_points = None          # coords of lattice points (direct)
        self.lat_points_cart = None     # coords of lattice points (cart)
        self.num_lat_points = None      # number of lattice points in perfect cell
        self.nsw = None                 # iteration number of md (nsw in incar)
        self.coords_on_lat = None       # trajectory matched on lattice point (cart): (num_target, num_step, 3)
        self.num_step = None            # nsw/interval
        self.occupy = None              # index of occupied site: (num_target, num_step)
        self.coords_vac = None          # dictionary; coords of vacancy (cart)
        self.vac_idx = None
        self.atom_species = None
        self.num_species = None
        self.num_atoms = None
        self.position = None
        self.idx_target = None          # index of taget in position
        self.trace_lines = None         # trace arrows
        self.cmap = ['b', 'g', 'r', 'gray', 'm', 'darkorange', 'c', 'indigo', 'brown']

        # read poscar and xdatcar
        self.read_xdatcar()
        self.read_poscar()
        self.coords_on_lattice()

        self.count = 0                  # number of atoms before target
        for spec in range(self.idx_target):
            self.count += self.position[spec]['num']   

        self.find_vacancy()
        self.get_trace_lines()

    def read_xdatcar(self):
        # read xdatcar
        lines = np.array([line.strip() for line in open(self.xdatcar)])
        self.lattice = np.array([line.split() for line in lines[2:5]], dtype=float)*float(lines[1])
        self.atom_species = np.array(lines[5].split())
        self.num_species = len(self.atom_species)
        self.num_atoms = np.array(lines[6].split(), dtype=int)
        num_tot = np.sum(self.num_atoms)
        self.nsw = int((lines.shape[0]-7)/(1+num_tot))
        self.num_step = int(self.nsw / self.interval)

        # save data
        self.position = []
        for spec in range(len(self.atom_species)):
            dic = {}
            # save basic data
            dic['species'] = self.atom_species[spec]
            dic['num'] = self.num_atoms[spec]
            if dic['species'] == self.target:
                self.idx_target = spec
            # get averaged coords
            coords_avg = np.zeros((dic['num'], self.num_step, 3))   # to save averaged coords
            traj_cart = np.zeros((dic['num'], self.nsw, 3))         # to save all coords
            for i in range(dic['num']):
                idx = np.sum(self.num_atoms[:spec]) + i
                # extract coords of each atom
                coords = np.array([line.split() for line in lines[(8+idx):(lines.shape[0]+1):(num_tot+1)]], dtype=float)
                # displacement
                displacement = np.zeros_like(coords)
                displacement[0,:] = 0
                displacement[1:,:] = np.diff(coords, axis=0)
                # correction for periodic boundary condition
                displacement[displacement>0.5] -= 1.0
                displacement[displacement<-0.5] += 1.0
                displacement = np.cumsum(displacement, axis=0)
                coords = coords[0] + displacement
                # covert to cartesian coords
                traj_cart[i] = np.dot(coords, self.lattice)
                # average coords at each interval
                for j in range(self.num_step):
                    mean_coord = np.average(coords[j*self.interval:(j+1)*self.interval], axis=0)
                    # wrap back into cell
                    mean_coord = mean_coord - np.floor(mean_coord)
                    coords_avg[i][j] = mean_coord
            coords_avg_cart = np.dot(coords_avg, self.lattice)
            dic['traj'] = traj_cart             # contains all coords, cart, (dic['num'], self.nsw, 3)
            dic['coords'] = coords_avg          # contains averaged coords, direct, (dic['num'], self.num_step, 3)
            dic['coods_cart'] = coords_avg_cart # contains averaged coords, cart, (dic['num'], self.num_step, 3)
            self.position += [dic]

    def read_poscar(self):
        # read poscar
        lines = np.array([line.strip() for line in open(self.poscar)])
        atom_species = np.array(lines[5].split()) 
        num_atoms = np.array(lines[6].split(), dtype=int)
        # get index of target atom
        idx = np.where(atom_species == self.target)[0][0]
        self.num_lat_points = num_atoms[idx]
        coords= lines[num_atoms[:idx].sum()+8:num_atoms[:idx+1].sum()+8]
        self.lat_points = np.array([line.split()[:3] for line in coords],dtype=float)
        self.lat_points_cart = np.dot(self.lat_points, self.lattice)

    def save_trajectory(self, interval_traj, foldername='traj', prefix='./'):
        if prefix[-1] != '/':
            prefix = prefix+'/'
        if not(os.path.isdir(prefix+foldername)):
            os.mkdir(prefix+foldername)

        for i in tqdm(range(self.position[self.idx_target]['num']), desc='make traj...'):
            idx = i + self.count + 1
            traj = self.position[self.idx_target]['traj'][i][0:-1:interval_traj]
            fig = plt.figure()
            ax = fig.add_subplot(111, projection = '3d')     
            # plot lattice and lattice points
            self.plot_lattice(ax)
            # plot trajectory
            ax.plot(*traj.T, 'b-', marker=None)
            ax.scatter(*traj[0], color='red')
            ax.scatter(*traj[-1], color='red', marker='x')
            # show and save plot
            filename = f'traj_{idx}.png'
            plt.title(f"Atom index = {idx}")
            plt.savefig(prefix+f"{foldername}/{filename}", format='png')
            plt.close()
            
    def distance_pbc(self, coord1, coord2):
        "coord1 and coord2 are in direct"
        "return : cartesian distance"
        distance = coord1 - coord2
        distance[distance>0.5] -= 1.0
        distance[distance<-0.5] += 1.0
        if coord1.ndim == 1:
            return np.sqrt(np.sum(np.dot(distance, self.lattice)**2))
        else:
            return np.sqrt(np.sum(np.dot(distance, self.lattice)**2,axis=1))

    def coords_on_lattice(self):
        self.coords_on_lat = np.zeros_like(self.position[self.idx_target]['coords'])
        self.occupy = np.zeros(self.coords_on_lat.shape[:2], dtype=int)
        for i in range(self.position[self.idx_target]['num']):
            for j, coord in enumerate(self.position[self.idx_target]['coords'][i]):
                distance = self.distance_pbc(self.lat_points, coord)
                site_occ = np.argmin(distance)
                self.coords_on_lat[i][j] = self.lat_points_cart[site_occ]
                self.occupy[i][j] = int(site_occ)

    def find_vacancy(self):
        # find candidate for vacancy
        self.coords_vac = {}
        self.vac_idx = {}
        for step in range (self.num_step):
            site_unocc = []
            # find lattice point which are not occupied
            for idx in range(self.num_lat_points):
                if not(idx in self.occupy[:,step]):
                    site_unocc += [idx]
            self.vac_idx[step] = site_unocc
            self.coords_vac[step] = self.lat_points_cart[site_unocc]

    def show_vac_init(self, label=False):
        self.show_vac(step=0, label=label)
    
    def show_vac(self, step, label=False):
        if self.coords_vac is None:
            self.find_vacancy(method='all')
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        # plot lattice and lattice points
        self.plot_lattice(ax, label=label)
        # plot initial vacancies
        for i, coord in enumerate(self.coords_vac[step]):
            ax.plot(*coord.T, color='yellow', 
                    marker='o', linestyle='none', markersize=8, alpha=0.8, zorder=1)
            ax.text(*coord.T, s=f"{self.vac_idx[step][i]}", weight='bold')
        plt.show()
            
    def plot_lattice(self, ax, label=False):
        coord_origin = np.zeros([1,3])
        edge = np.concatenate(
            (coord_origin, self.lattice[0].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            (coord_origin, self.lattice[1].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            (coord_origin, self.lattice[2].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[0]+self.lattice[1]).reshape(1,3), 
             self.lattice[0].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[0]+self.lattice[1]).reshape(1,3), 
             self.lattice[1].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[1]+self.lattice[2]).reshape(1,3), 
             self.lattice[1].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[1]+self.lattice[2]).reshape(1,3), 
             self.lattice[2].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[2]+self.lattice[0]).reshape(1,3), 
             self.lattice[2].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[2]+self.lattice[0]).reshape(1,3), 
             self.lattice[0].reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[0]+self.lattice[1]+self.lattice[2]).reshape(1,3), 
             (self.lattice[0]+self.lattice[1]).reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[0]+self.lattice[1]+self.lattice[2]).reshape(1,3), 
             (self.lattice[1]+self.lattice[2]).reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        edge = np.concatenate(
            ((self.lattice[0]+self.lattice[1]+self.lattice[2]).reshape(1,3), 
             (self.lattice[2]+self.lattice[0]).reshape(1,3)), axis=0).T
        ax.plot(edge[0], edge[1], edge[2], 'k-', marker='o')
        
        # plot lattice points
        ax.scatter(*self.lat_points_cart.T, facecolor='none', edgecolors='k', alpha=0.8)
        if label:
            for i, coord in enumerate(self.lat_points_cart):
                ax.text(*coord.T, s=f"{i}", fontsize='xx-small')
        
        # axis label
        ax.set_xlabel('x (Å)')
        ax.set_ylabel('y (Å)')
        ax.set_zlabel('z (Å)')

    def save_gif_PIL(self, filename, files, fps=5, loop=0):
        imgs = [Image.open(file) for file in files]
        imgs[0].save(fp=filename, format='GIF', append_images=imgs[1:], 
                     save_all=True, duration=int(1000/fps), loop=loop)

    def plot_trajectory(self, index, interval_traj=100, label=False):
        "TODO: Check!! something seems to be wrong!!" 
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        # plot lattic and lattice points
        self.plot_lattice(ax, label=label)
        # plot trajectories
        for i, idx in enumerate(index):
            idx = idx - self.count - 1
            ax.scatter(*self.coords_on_lat[idx].T, 
                       facecolor=self.cmap[i%len(self.cmap)], edgecolor='none', alpha=0.8, label=f"{index[i]}")
            ax.plot(*self.position[self.idx_target]['traj'][idx][0:-1:interval_traj].T, 
                    c=self.cmap[i%len(self.cmap)], marker=None, alpha=0.6)
        plt.legend()
        plt.show()

    def animation(self, index, steps='all', vac=True, 
                  gif=True, filename="trajectory.gif", foldername='gif',
                  update_alpha=0.75, potim=2, fps=5, loop=0, dpi=300, 
                  legend=False, label=False, prefix='./'):
        """
        potim is required to convert step to time
        index : from VESTA of defective poscar
        """
        if prefix[-1] != '/':
            prefix = prefix+'/'
        if not(os.path.isdir(prefix+foldername)):
            os.mkdir(prefix+foldername)

        step_to_save = range(self.num_step) if steps=='all' else steps
        files = []
        for step in tqdm(step_to_save, desc='making gif...'):
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            # plot lattic and lattice points
            self.plot_lattice(ax, label=label)
            # plot points
            for i, idx in enumerate(index):
                idx = idx - self.count - 1
                ax.scatter(*self.coords_on_lat[idx][step].T, 
                           facecolor=self.cmap[i%len(self.cmap)], edgecolor='none', alpha=0.8, label=f"{index[i]}")
            # plot trace lines
            alpha = 1
            for i in reversed(range(step)):
                for arrow in self.trace_lines[i]:
                    arrow_prop_dict = dict(mutation_scale=10, arrowstyle='->', color=arrow['c'],
                                           alpha = alpha, shrinkA=0, shrinkB=0)
                    draw_arrow = Arrow3D(*arrow['p'].T, **arrow_prop_dict)
                    ax.add_artist(draw_arrow)
                alpha *= update_alpha
            # plot vacancy
            if vac:
                if self.coords_vac is None:
                    self.find_vacancy()
                ax.plot(*self.coords_vac[step].T, color='yellow', marker='o', linestyle='none', 
                        markersize=8, alpha=0.8, zorder=1)
            # decorate plot
            time = step * self.interval * potim / 1000 # ps
            time_tot = self.nsw * potim / 1000 # ps
            plt.title("(%.2f/%.2f) ps, (%d/%d) step"%(time, time_tot, step, self.num_step))
            if legend:
                plt.legend()
            # save plots
            file = prefix+f"{foldername}/traj_{step}.png"
            files.append(file)
            plt.savefig(file, dpi=dpi)
            plt.close()
        if gif:
            print("Making GIF file....")
            self.save_gif_PIL(filename=prefix+filename, files=files, fps=fps, loop=loop)
            print("GIF file was generated.")
    
    def get_trace_lines(self):
        self.trace_lines = {}
        atoms = np.arange(self.occupy.shape[0])
        for step in range(1, self.num_step):
            arrows = []
            for atom in atoms:
                coord_now = self.lat_points[self.occupy[atom][step]]
                coord_before = self.lat_points[self.occupy[atom][step-1]]
                distance = self.distance_pbc(coord_now, coord_before)
                if distance > 0.001:
                    arrow = {}
                    arrow['p'] = np.vstack((self.coords_on_lat[atom][step-1],self.coords_on_lat[atom][step]))
                    arrow['c'] = self.cmap[atom%len(self.cmap)]
                    arrow['lat_points'] = [self.occupy[atom][step-1], self.occupy[atom][step]]
                    arrows += [arrow]
            self.trace_lines[step-1] = arrows

    def get_poscar(self, step, vac=False):
        "vac=True to include vacancy"
        "vac is labelled by XX"
        filename = f"POSCAR_{step}"
        with open(filename, 'w') as f:
            f.write(f"step_{step}\n")
            f.write("1.0\n")
            for lat in self.lattice:
                f.write("%.6f %.6f %.6f\n"%(lat[0], lat[1], lat[2]))
            for dic in self.position:
                f.write(f"{dic['species']} ")
            if vac:
                f.write("XX")
            f.write("\n")
            for dic in self.position:
                f.write(f"{dic['num']} ")
            if vac:
                f.write(f"{len(self.vac_idx[step])}")
            f.write("\n")
            f.write("Direct\n")
            for dic in self.position:
                for coord in dic['coords'][:,step,:]:
                    f.write("%.6f %.6f %.6f\n"%(coord[0], coord[1], coord[2]))
            if vac:
                for idx in self.vac_idx[step]:
                    coord = self.lat_points[idx]
                    f.write("%.6f %.6f %.6f\n"%(coord[0], coord[1], coord[2]))
                
    def show_poscar(self, step=None, filename=None, vac=False):
        "recieve step or filename"
        if step is not None:
            self.get_poscar(step=step, vac=vac)
            filename = f"POSCAR_{step}"
        structure = read_vasp(filename)
        view(structure)
        
    def trace_movement(self, index=[], step=[], foldername='trace', prefix='./',
                       vac=True, label=False, potim=2, dpi=300):
        """
        index: number of lattice points at the first element of step array.
        """
        if prefix[-1] != '/':
            prefix = prefix+'/'
        if not(os.path.isdir(prefix+foldername)):
            os.mkdir(prefix+foldername)

        # obtain atom numbers
        atom_idx = []
        for idx in index:
            atom_idx += [np.argmax(self.occupy[:,step[0]]==idx)]
        print(f"atom index : {atom_idx}")
        
        check_first = True  # check whether the first loop
        points_init = []
        for s in tqdm(step):
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            self.plot_lattice(ax, label=label)
            for i, idx in enumerate(atom_idx):
                # plot points
                ax.scatter(*self.coords_on_lat[idx][s].T, facecolor=self.cmap[i%len(self.cmap)],
                           edgecolor='none', alpha=0.8, label=f"{index[i]}")
                # save initial postions
                if check_first:
                    point_init = {}
                    point_init['p'] = self.coords_on_lat[idx][s]
                    point_init['c'] = self.cmap[i%len(self.cmap)]
                    points_init += [point_init]
            check_first = False
            # plot trajectory arrow
            alpha = 1
            for line in self.trace_lines[s-1]:
                arrow_prop_dict = dict(mutation_scale=10, arrowstyle='->', color='k',
                                       alpha = alpha, shrinkA=0, shrinkB=0)
                arrow = Arrow3D(*line['p'].T, **arrow_prop_dict)
                ax.add_artist(arrow)
            # show the initial positions
            for point in points_init:
                ax.plot(*point['p'].T, c=point['c'], marker='o', linestyle='none', 
                        markersize=10, alpha=0.4, zorder=0)
            if vac:
                ax.plot(*self.coords_vac[s].T, color='yellow', marker='o', linestyle='none', 
                        markersize=8, alpha=0.8, zorder=1)
            time = s * self.interval * potim / 1000
            time_tot = self.nsw * potim / 1000
            plt.title("(%.2f/%.2f) ps, (%d/%d) step"%(time, time_tot, s, self.num_step))
            file = f"{prefix+foldername}/trace_{s}.png"
            plt.savefig(file, dpi=dpi)
            plt.close()

    def check_num_vac(self):
        multi_vac = []
        for step, idx in enumerate(self.vac_idx.values()):
            num_vac = len(idx)
            if num_vac != 1:
                multi_vac += [step]
        if len(multi_vac) == 0:
            print("There is only one vacancy")
        else:
            print("There are mutiple vacancies in some steps !!!")
            for s in multi_vac:
                print(s, end=", ")
    
    def update_vac_site(self, step, idx):
        self.vac_idx[step] = [idx]
        self.coords_vac[step] = np.array([self.lat_points_cart[idx]])

    def trace_vacancy(self, start=1):
        """
        tracing vacancy from (start) step
        """
        trace_lines = self.trace_lines
        vac_site = self.vac_idx[0][0]
        for step in range(start, self.num_step):
            # when only one vacancy exist
            if len(self.vac_idx[step]) == 1:
                vac_site = self.vac_idx[step][0]
                self.update_vac_site(step, vac_site)
                continue
            # when multiple vacancies exsit
            # when vacancy is stationary
            if vac_site in self.vac_idx[step]:
                self.update_vac_site(step, vac_site)
                continue
            # when vacancy moves
            # find connected points with vacancy
            points = [vac_site]
            while True:
                check1 = len(points)
                for dic in trace_lines[step-1]:
                    if len(list(set(points) & set(dic['lat_points']))) == 1:
                        points += dic['lat_points']
                        points = list(set(points))
                check2 = len(points)
                if check1 == check2:
                    break
            site = list(set(points) & set(self.vac_idx[step]))
            if len(site) == 1:
                vac_site = site[0]
                self.update_vac_site(step, vac_site)
            elif len(site) == 0:
                print("There is no connected site.")       
                print(f"Find the vacancy site for your self. (step: {step})")
                break
            else:
                print("There are multiple candidates.")       
                print(f"Find the vacancy site for your self. (step: {step})")
                break

In [81]:
class HfO2_Analyzer:
    def __init__(self, traj):
        self.traj = traj
        self.traj_backup = traj

        # diffusion paths in HfO2
        self.path_cn3 = []
        self.path_cn4 = []
        self.diffusion_path()

        # lattice points with cn
        self.lat_points = []
        self.lattice_points()

        # path of vacancy
        self.path_vac = []
        self.path_of_vacancy()

        self.idx_U = []
    
    def diffusion_path(self):
        """
        paths from neb calculations
        """
        d_cn3 = [2.54239, 2.57364, 2.78548, 2.83698, 2.93743, 2.96476, 2.98909]
        Ea_cn3 = [0.74, 0.84, 0.85, 1.35, 1.91, 2.07, 2.01]
        cn_cn3 = [4, 3, 3, 3, 4, 4, 4]
        for i in range(7):
            path = {}
            path['name'] = f"A{i+1}"
            path['d'] = d_cn3[i]
            path['Ea'] = Ea_cn3[i]
            path['cn'] = cn_cn3[i]
            self.path_cn3 += [path]

        d_cn4 = [2.54239, 2.57563, 2.6619, 2.72384, 2.93743, 2.96476, 2.98909]
        Ea_cn4 = [0.08, 0.32, 0.86, 0.98, 1.25, 1.42, 1.36]
        cn_cn4 = [3, 4, 4, 4, 3, 3, 3]
        for i in range(7):
            path = {}
            path['name'] = f"B{i+1}"
            path['d'] = d_cn4[i]
            path['Ea'] = Ea_cn4[i]
            path['cn'] = cn_cn4[i]
            self.path_cn4 += [path]

    def lattice_points(self):
        for coord in self.traj.lat_points:
            dic = {}
            dic['coord'] = coord
            if 0.13796 < coord[0] < 0.36204 or 0.63796 < coord[0] < 0.86204:
                dic['cn'] = 4
            else:
                dic['cn'] = 3
            self.lat_points += [dic]
    
    def path_of_vacancy(self):
        for i in range(self.traj.num_step-1):
            coord1 = self.lat_points[self.traj.vac_idx[i][0]]['coord']
            coord2 = self.lat_points[self.traj.vac_idx[i+1][0]]['coord']
            d = self.traj.distance_pbc(coord1, coord2)
            # check whether vacancy moves
            if d > 0.1:
                dic_path = {}
                dic_path['cn'] = self.lat_points[self.traj.vac_idx[i][0]]['cn']
                name, Ea, err = self.get_path(dic_path['cn'], d)
                dic_path['path_name'] = [name]
                dic_path['Ea'] = Ea
                dic_path['err'] = err
                dic_path['step'] = i+1
                self.path_vac += [dic_path]
            
    def get_path(self, cn_init, d):
        """
        cn_init : coordination number of vacancy at initial state
        d : distance of the path
        """
        if not(cn_init in [3, 4]):
            print("coordination number in HfO2 should be 3 or 4.")
            exit()
        paths = self.path_cn3 if cn_init == 3 else self.path_cn4
        err_min, idx_min = 100, None
        for i, path in enumerate(paths):
            err = abs(d - path['d'])
            if err < err_min:
                err_min = err
                idx_min = i
        return paths[idx_min]['name'], paths[idx_min]['Ea'], err_min
    
    def print_path(self, disp=True, short=False):
        if len(self.path_vac) == 0:
            print("no movement of vacancy.")
            return
        Ea_max = 0
        path_list = []
        self.idx_U = []
        for i, dic_path in enumerate(self.path_vac):
            if dic_path['err'] < 0.001:
                if disp:
                    print(f"[{i}] {dic_path['path_name']}, {dic_path['Ea']} eV (step: {dic_path['step']})")
                path_list += dic_path['path_name']
                if dic_path['Ea'] > Ea_max:
                    Ea_max = dic_path['Ea']
            else:
                if disp:
                    print(f"[{i}] Unknown, err={dic_path['err']} (step: {dic_path['step']})")
                path_list += ['U']
                self.idx_U += [i]
        if disp:
            print("")
            for name in path_list:
                print(name, end=" ")
            print("")
            print(f"num_path: {len(path_list)}")
            print(f"Ea: {Ea_max}")
            print(f"number of U : {len(self.idx_U)}")
            print(f"step of U :", end = " ")
            for idx in self.idx_U:
                print(self.path_vac[idx]['step'], end= " ")
            print("")
        elif short:
            for name in path_list:
                print(name, end=" ")
            print("\n"+f"num_path: {len(path_list)}")
            print(f"number of U : {len(self.idx_U)}")

    def path_finder(self, paths, p_init, p_goal):
        """
        find sequential paths connection p_init and p_goal
        """
        answer = [p_init]
        while True:
            if answer[-1] == p_goal:
                return answer
            intersect = []
            for i, path in enumerate(paths):
                if path[0] == p_init:
                    intersect += [i]
            if len(intersect) == 1:
                p_init = paths[intersect[0]][1]
                answer += [p_init]
            elif len(intersect) == 0:
                return []
            else:
                for i in intersect:
                    answer += self.path_finder(paths, paths[i][1], p_goal)
                if answer[-1] != p_goal:
                    return []
    
    def decompose_path(self, idx):
        """
        idx : index in self.path_vac
        """
        step = self.path_vac[idx]['step']
        arrows = np.zeros((len(self.traj.trace_lines[step-1]), 2))
        for i, dic_line in enumerate(self.traj.trace_lines[step-1]):
            arrows[i] = dic_line['lat_points']
        # arrows = np.array(arrows, dtype=int)

        vac_now = self.traj.vac_idx[step][0]
        vac_before = self.traj.vac_idx[step-1][0]

        path = self.path_finder(arrows, vac_now, vac_before)
        path = np.array(path, dtype=int)
        return path
    
    def confine_path(self):
        for idx in self.idx_U:
            try:
                path = self.decompose_path(idx)
                path = np.flip(path)
            except:
                print(f"Error. index of path: {idx}")
                return
            if len(path) == 0:
                continue

            names = []
            Ea_max = 0
            err_max = 0
            for i in range(len(path)-1):
                coord1 = self.lat_points[path[i]]['coord']
                coord2 = self.lat_points[path[i+1]]['coord']
                cn = self.lat_points[path[i]]['cn']
                d = self.traj.distance_pbc(coord1, coord2)
                name, Ea, err = self.get_path(cn, d)
                if err > 0.001:
                    # hopping, but no matched distance
                    print(f"[{idx}] undefined distance. d={d}, points={path[i]}, {path[i+1]} (step: {self.path_vac[idx]['step']})")
                    names += ['U']
                else:
                    names += [name]
                    if Ea > Ea_max:
                        Ea_max = Ea
                if err > err_max:
                    err_max = err
            self.path_vac[idx]['path_name'] = names
            self.path_vac[idx]['Ea'] = Ea_max
            self.path_vac[idx]['err'] = err_max
        print("")

In [90]:
temp = 1200

for i in range(20):
    ensemble = format(i+1,'02')
    print(ensemble)

    prefix_data = '../xdatcar/'
    poscar = prefix_data+'POSCAR_SUPERCELL'
    xdatcar = prefix_data+f"xdatcar.{temp}K/XDATCAR_{ensemble}"
    traj = TrajectoryOnLattice(poscar=poscar, xdatcar=xdatcar, interval=50)
    # confine vacancy site
    traj.trace_vacancy(start=1)
    traj.check_num_vac()

    analyzer = HfO2_Analyzer(traj=traj)
    analyzer.print_path(disp=False)
    analyzer.confine_path()
    analyzer.print_path(disp=False, short=True)

01
There is only one vacancy

A1 B1 A1 B1 
num_path: 4
number of U : 0
02
There is only one vacancy
no movement of vacancy.

no movement of vacancy.
03
There is only one vacancy

A1 B2 B1 
num_path: 3
number of U : 0
04
There is only one vacancy
no movement of vacancy.

no movement of vacancy.
05
There is only one vacancy

A1 B1 
num_path: 2
number of U : 0
06
There is only one vacancy
no movement of vacancy.

no movement of vacancy.
07
There is only one vacancy

A1 B1 A3 A3 
num_path: 4
number of U : 0
08
There is only one vacancy

A1 B1 
num_path: 2
number of U : 0
09
There is only one vacancy

A1 B1 
num_path: 2
number of U : 0
10
There is only one vacancy
no movement of vacancy.

no movement of vacancy.
11
There is only one vacancy
no movement of vacancy.

no movement of vacancy.
12
There is only one vacancy
no movement of vacancy.

no movement of vacancy.
13
There is only one vacancy
no movement of vacancy.

no movement of vacancy.
14
There is only one vacancy

A1 B1 A1 B1 
num_pa

In [48]:
analyzer = HfO2_Analyzer(traj=traj)
analyzer.print_path(disp=False)
analyzer.confine_path()
analyzer.print_path(disp=True)
print("\n")


[0] ['A3'], 0.85 eV (step: 1)
[1] ['A3'], 0.85 eV (step: 2)
[2] ['A3'], 0.85 eV (step: 22)
[3] ['A3'], 0.85 eV (step: 29)
[4] ['A3'], 0.85 eV (step: 38)
[5] ['A1'], 0.74 eV (step: 45)
[6] ['B2'], 0.32 eV (step: 47)
[7] ['B1'], 0.08 eV (step: 51)
[8] ['A3'], 0.85 eV (step: 76)
[9] ['A3'], 0.85 eV (step: 89)
[10] ['A3'], 0.85 eV (step: 103)
[11] ['A3'], 0.85 eV (step: 107)
[12] ['A7'], 2.01 eV (step: 113)
[13] ['B2'], 0.32 eV (step: 127)
[14] ['B1'], 0.08 eV (step: 130)
[15] ['A3'], 0.85 eV (step: 137)
[16] ['A3'], 0.85 eV (step: 143)
[17] ['A3'], 0.85 eV (step: 144)
[18] ['A3'], 0.85 eV (step: 149)
[19] ['A3', 'A7', 'B1'], 2.01 eV (step: 150)
[20] ['A1'], 0.74 eV (step: 154)
[21] ['B1'], 0.08 eV (step: 157)
[22] ['A3'], 0.85 eV (step: 159)
[23] ['A3'], 0.85 eV (step: 161)
[24] ['A4'], 1.35 eV (step: 162)
[25] ['A2'], 0.84 eV (step: 165)
[26] ['A1'], 0.74 eV (step: 175)
[27] ['B1'], 0.08 eV (step: 176)
[28] ['A1'], 0.74 eV (step: 189)
[29] ['B1'], 0.08 eV (step: 190)
[30] Unknown, err=2

In [6]:
correct_vac_site = []
## 2200K
# correct_vac_site = [[127, 7]] # 01
# correct_vac_site = [[182, 60],[194, 45],[189, 28],[190, 28],[191, 28],[192, 53]] # 02

## 2300K
# correct_vac_site = [[34,54],[175,62]] # 09

## 2400K
# correct_vac_site = [[295, 4],[273, 12],[279, 56],[283, 54],[291, 33],[288, 12]] # 02
# correct_vac_site = [[142, 40], [156, 48], [180, 28], [113, 28]] # 05
# correct_vac_site = [[91, 28],[97,32],[128,50],[144,26],[96,61]] # 09
# correct_vac_site = [[92, 54],[125, 55],[165, 56],[175, 54],[194, 47],[208, 55],
#                     [220, 46],[236, 55],[243, 58],[290, 59],[74, 46],[77, 46],
#                     [90, 2],[91, 54],[72, 30],[193, 58],[219, 47],[75, 30]] # 10

## 2500K
# correct_vac_site = [[13,33],[234,53],[244,24]] # 01
# correct_vac_site = [[34,32],[51,15],[63,23],[81,42],[258,6],[275,46],[279,55]] # 02
# correct_vac_site = [[98,53],[142,33],[156,32],[173,61],[201,36],[213,41],[216,60],
#                     [217,61],[221,61],[223,36],[279,60],[286,36],[291,32],[294,15]] # 03
# correct_vac_site = [[14,33],[41,9],[129,5],[160,36],[164,33],[168,57],[167,16],
#                     [204,33],[251,39],[264,55],[271,2]] # 04
# correct_vac_site = [[101,42],[105,7],[170,54],[203,58],[207,58],[239,43],[269,50],
#                     [286,47],[288,27],[290,27]] # 05
# correct_vac_site = [[214,60],[240,21], [222,46], [226,46], [233,32]] # 06
# correct_vac_site = [[24,49],[46,50],[59,58],[99,51],[107,46],[163,29],[194,59]] # 07
# correct_vac_site = [[201,13]] # 08
# correct_vac_site = [[28,40],[126,12],[276,47]] # 09
# correct_vac_site = [[103,21],[108,26],[111,35],[226,29]] # 10

for step, idx in correct_vac_site:
    traj.update_vac_site(step=step, idx=idx)
    traj.trace_vacancy(start=step)
print("")
traj.check_num_vac()


There is only one vacancy


In [22]:
analyzer = HfO2_Analyzer(traj=traj)
analyzer.print_path(disp=False)
analyzer.confine_path()
analyzer.print_path(disp=True)


[0] ['A1'], 0.74 eV (step: 190)
[1] ['B1'], 0.08 eV (step: 191)

A1 B1 
num_path: 2
Ea: 0.74
number of U : 0
step of U : 


In [7]:
# animations
if not(os.path.isdir(f"./{temp}K")):
    os.mkdir(f"./{temp}K")
if not(os.path.isdir(f"./{temp}K/{ensemble}")):
    os.mkdir(f"./{temp}K/{ensemble}")
    
steps = 'all'
# steps = range(210, 230)

traj.animation(index=np.arange(33, 96),
               steps = steps,
               gif=False,
               label = True,
               update_alpha=0,
               fps=20,
               foldername='gif_label', 
               filename='trajectory_label.gif',
               prefix=f"./{temp}K/{ensemble}")

making gif...:   0%|          | 0/300 [00:00<?, ?it/s]

In [48]:
def distance_Hf(step):
    Hf_1st = np.array([5, 7, 29, 31, 6, 8, 30, 32]) - 1
    Hf_2nd = np.array([13, 15, 21, 23, 14, 16, 22, 24]) - 1
    Hf_3rd = np.array([1, 3, 25, 27, 2, 4, 26, 28]) - 1
    Hf_4th = np.array([9, 11, 17, 19, 10, 12, 18, 20]) - 1

    idx_Hf = np.zeros((4, len(Hf_1st)))
    idx_Hf[0] = Hf_1st
    idx_Hf[1] = Hf_2nd
    idx_Hf[2] = Hf_3rd
    idx_Hf[3] = Hf_4th
    idx_Hf = np.array(idx_Hf, dtype=int)

    coords_Hf = traj.position[0]['coords'][:,step,0]

    a_coord_avg = np.zeros(4)
    for i in range(4):
        a_coord_avg[i] = np.average(coords_Hf[idx_Hf[i]])

    gap_layer = -np.diff(a_coord_avg)
    gap_layer = np.concatenate((gap_layer, [1-(a_coord_avg[0]-a_coord_avg[-1])]), axis=0)

    gap_cn3 = np.average(gap_layer[[1,3]])
    gap_cn4 = np.average(gap_layer[[0,2]])

    if gap_cn3 > gap_cn4:
        # Not Flipped
        return 0
    else:
        # Flipped
        return 1

In [63]:
temp = 2500
ensembles = [format(i+1,'02') for i in range(10)]

for ensemble in ensembles:
    prefix_data = '../xdatcar/'
    poscar = prefix_data+'POSCAR_SUPERCELL'
    xdatcar = prefix_data+f"xdatcar.{temp}K/XDATCAR_{ensemble}"
    traj = TrajectoryOnLattice(poscar=poscar, xdatcar=xdatcar, interval=50)

    step_flip = []
    check_flip_before = distance_Hf(0)
    for i in range(traj.num_step):
        check_flip = distance_Hf(i)
        if check_flip != check_flip_before:
            step_flip += [i]
        check_flip_before = check_flip
    print(f"[{ensemble}] steps flip occurs : {step_flip}")



[01] steps flip occurs : [9, 11, 12, 15, 16, 17, 18, 74, 76, 80, 93, 94, 166, 168, 218, 219, 220, 223, 227, 228, 230, 236, 238, 241, 242, 251, 253, 256]
[02] steps flip occurs : [37, 38, 39, 40, 44, 47, 50, 53, 54, 68, 69, 146, 149, 160, 161, 167, 168, 220, 221, 234, 246, 247, 256, 266, 276, 277, 279, 280, 281, 290, 291]
[03] steps flip occurs : [10, 11, 69, 73, 78, 88, 91, 94, 103, 105, 106, 140, 296]
[04] steps flip occurs : [20, 21, 22, 23, 25, 27, 38, 39, 40, 45, 46, 49, 51, 52, 111, 132, 133, 139, 140, 143, 146, 157, 170, 171, 224, 235, 236, 245, 246, 249, 250, 253, 257, 258, 259, 262]
[05] steps flip occurs : [93, 94, 100, 104, 105, 106, 114, 174, 188, 203]
[06] steps flip occurs : [207, 217, 218, 223, 225, 226, 233, 234, 239, 289, 290, 292, 293, 294, 295]
[07] steps flip occurs : [28, 38, 46, 50, 52, 57, 59, 67, 69, 72, 76, 78, 79, 87, 90, 91, 92, 93, 105, 108, 109, 110, 111, 160, 162, 167, 175, 178, 180, 182, 184, 187, 188, 198, 201, 217]
[08] steps flip occurs : [15, 18, 71, 7

In [78]:
temp = 2200
ensemble= '02'

prefix_data = '../xdatcar/'
poscar = prefix_data+'POSCAR_SUPERCELL'
xdatcar = prefix_data+f"xdatcar.{temp}K/XDATCAR_{ensemble}"
traj = TrajectoryOnLattice(poscar=poscar, xdatcar=xdatcar, interval=50)

step_flip = []
flips = []
check_flip_before = distance_Hf(0)
for i in range(traj.num_step):
    check_flip = distance_Hf(i)
    flips += [check_flip]
    if check_flip != check_flip_before:
        step_flip += [i]
    check_flip_before = check_flip
print(f"steps flip occurs : {step_flip}")


steps flip occurs : [175, 180, 181, 182, 183, 184, 189, 197, 207, 208, 212, 213, 218, 224, 227]


In [79]:
for i, flip in enumerate(flips):
    print(f"{[i]} {flip}")

[0] 0
[1] 0
[2] 0
[3] 0
[4] 0
[5] 0
[6] 0
[7] 0
[8] 0
[9] 0
[10] 0
[11] 0
[12] 0
[13] 0
[14] 0
[15] 0
[16] 0
[17] 0
[18] 0
[19] 0
[20] 0
[21] 0
[22] 0
[23] 0
[24] 0
[25] 0
[26] 0
[27] 0
[28] 0
[29] 0
[30] 0
[31] 0
[32] 0
[33] 0
[34] 0
[35] 0
[36] 0
[37] 0
[38] 0
[39] 0
[40] 0
[41] 0
[42] 0
[43] 0
[44] 0
[45] 0
[46] 0
[47] 0
[48] 0
[49] 0
[50] 0
[51] 0
[52] 0
[53] 0
[54] 0
[55] 0
[56] 0
[57] 0
[58] 0
[59] 0
[60] 0
[61] 0
[62] 0
[63] 0
[64] 0
[65] 0
[66] 0
[67] 0
[68] 0
[69] 0
[70] 0
[71] 0
[72] 0
[73] 0
[74] 0
[75] 0
[76] 0
[77] 0
[78] 0
[79] 0
[80] 0
[81] 0
[82] 0
[83] 0
[84] 0
[85] 0
[86] 0
[87] 0
[88] 0
[89] 0
[90] 0
[91] 0
[92] 0
[93] 0
[94] 0
[95] 0
[96] 0
[97] 0
[98] 0
[99] 0
[100] 0
[101] 0
[102] 0
[103] 0
[104] 0
[105] 0
[106] 0
[107] 0
[108] 0
[109] 0
[110] 0
[111] 0
[112] 0
[113] 0
[114] 0
[115] 0
[116] 0
[117] 0
[118] 0
[119] 0
[120] 0
[121] 0
[122] 0
[123] 0
[124] 0
[125] 0
[126] 0
[127] 0
[128] 0
[129] 0
[130] 0
[131] 0
[132] 0
[133] 0
[134] 0
[135] 0
[136] 0
[137] 0
[138] 

In [80]:
# poscar generation
step = range(175,227)
for s in step:
    traj.get_poscar(step=s, vac=True)

In [ ]:
# if not(os.path.isdir(f"./{temp}K")):
#     os.mkdir(f"./{temp}K")
# if not(os.path.isdir(f"./{temp}K/{ensemble}")):
#     os.mkdir(f"./{temp}K/{ensemble}")
# traj.trace_movement(index=[46, 54, 47, 55], step=range(61, 80), prefix=f"./{temp}K/{ensemble}", label=False)